In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Python version: 3.6


import os
import copy
import time
import pickle
import numpy as np
from tqdm import tqdm

import torch
from tensorboardX import SummaryWriter

import argparse
from options import args_parser
from update import LocalUpdate, test_inference
from models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar
from utils import get_dataset, average_weights, exp_details




In [2]:
# if __name__ == '__main__':
start_time = time.time()

# define paths
path_project = os.path.abspath('..')
logger = SummaryWriter('../logs')
print('here3')

args = args_parser()
print('here2')
exp_details(args)
print('here1')
device = 'cuda' if args.gpu else 'cpu'

if args.gpu:
    torch.cuda.set_device(args.gpu)

    
    
# load dataset and user groups
train_dataset, test_dataset, user_groups = get_dataset(args)

# BUILD MODEL
if args.model == 'cnn':
    # Convolutional neural netork
    if args.dataset == 'mnist':
        global_model = CNNMnist(args=args)
    elif args.dataset == 'fmnist':
        global_model = CNNFashion_Mnist(args=args)
    elif args.dataset == 'cifar':
        global_model = CNNCifar(args=args)

elif args.model == 'mlp':
    # Multi-layer preceptron
    img_size = train_dataset[0][0].shape
    len_in = 1
    for x in img_size:
        len_in *= x
        global_model = MLP(dim_in=len_in, dim_hidden=64,
                           dim_out=args.num_classes)
else:
    print('Error: unrecognized model')



here3
here2

Experimental details:
    Model     : mlp
    Optimizer : sgd
    Learning  : 0.01
    Global Rounds   : 10

    Federated parameters:
    IID
    Fraction of users  : 0.1
    Local Batch size   : 10
    Local Epochs       : 10

here1


In [3]:
from matplotlib import pyplot as plt


In [4]:

# num_users=100
# test=np.random.exponential(scale=1.0, size=num_users)
# print(test)
# plt.hist(test)
# plt.show()
# norm=np.random.normal(loc=1.19865, scale=1.0, size=num_users)
# print(norm)
# plt.plot(np.arange(norm.size),norm)
# plt.show()

In [5]:
# (-0.4242+2.8215)/2

In [6]:
# def gen_response_time(num_users=args.num_users):
#     np.random.exponential(scale=1.0, size=num_users)
    

In [7]:
# Set the model to train and send it to device.
global_model.to(device)
global_model.train()
print(global_model)

# copy weights
global_weights = global_model.state_dict()

# Training
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0
res_times_epochs = []
selected_user_idxs_epochs=[]
average_time_epochs=[]
test_accs_epochs=[]
noise_levels_epochs=[]
seed=20
max_time_epochs=[]
# np.random.seed(seed)

for epoch in tqdm(range(args.epochs)):
    test_accs=[]
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')
    res_times=np.random.exponential(scale=10, size=args.num_users)
    res_times_epochs.append(res_times)
    print(res_times[:10])
    noise_levels = np.random.uniform(low=0, high=1, size=args.num_users) # noise level for norm dist, 0 mean, this is var
    noise_levels_epochs.append(noise_levels)
    global_model.train()
    # m = max(int(args.frac * args.num_users), 1)
    m = 10

    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    selected_user_idxs_epochs.append(idxs_users)
    for idx in idxs_users:
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger, noise_level=noise_levels[idx], seed=seed)
        w, loss, test_acc = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))
        test_accs.append(test_acc)
    test_accs_epochs.append(test_accs)
    
    average_time=0
    max_time=0
    for idx in idxs_users:
        average_time+=res_times[idx]
        max_time = max(max_time, res_times[idx])
    
    average_time = average_time/len(idxs_users)
    average_time_epochs.append(average_time)
    max_time_epochs.append(max_time)
    # update global weights
    global_weights = average_weights(local_weights)

    # update global weights
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    global_model.eval()
    for c in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger, noise_level=noise_levels[idx], seed=seed)
        acc, loss = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    train_accuracy.append(sum(list_acc)/len(list_acc))

    # print global training loss after every 'i' rounds
    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))



MLP(
  (layer_input): Linear(in_features=784, out_features=64, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=64, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


  0%|                                                    | 0/10 [00:00<?, ?it/s]


 | Global Training Round : 1 |

[19.43664992  5.24561408  1.99752981  9.25670477 12.28232634  0.40932905
  4.7329129  15.18004028 13.8801674   0.12217642]


/Users/xin/Developer/CMPUT675_Project/src/update.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)


test_accu: 0.5833333333333334  test loss: -4.52642460167408
test_accu: 0.5166666666666667  test loss: -3.625602275133133
test_accu: 0.5666666666666667  test loss: -4.4148664474487305
test_accu: 0.5166666666666667  test loss: -3.9206603094935417
test_accu: 0.6666666666666666  test loss: -4.556306108832359
test_accu: 0.6  test loss: -3.8230721950531006
test_accu: 0.7  test loss: -4.624597579240799
test_accu: 0.6  test loss: -4.609623461961746
test_accu: 0.5666666666666667  test loss: -3.983491227030754
test_accu: 0.6333333333333333  test loss: -5.078565552830696


...

 
Avg Training Stats after 8 global rounds:
Training Loss : -0.6280152971854356
Train Accuracy: 65.00% 


 | Global Training Round : 9 |

[10.28519426  7.11673776  2.16724851  6.04657988  9.05542301  0.82745124
 29.05196837  0.04845718  6.491628   13.91748486]
test_accu: 0.8  test loss: -7.675090402364731
test_accu: 0.6833333333333333  test loss: -6.730413913726807
test_accu: 0.6166666666666667  test loss: -6.291409030556679
test_accu: 0.65  test loss: -6.398808404803276
test_accu: 0.6833333333333333  test loss: -6.5837893187999725
test_accu: 0.7666666666666667  test loss: -7.3355569541454315
test_accu: 0.7166666666666667  test loss: -6.892765134572983
test_accu: 0.7333333333333333  test loss: -6.975696086883545
test_accu: 0.75  test loss: -7.165067136287689
test_accu: 0.8166666666666667  test loss: -7.884669840335846


 90%|███████████████████████████████████████▌    | 9/10 [02:42<00:18, 18.63s/it]


 | Global Training Round : 10 |

[ 2.61342796  2.98443309  1.27011022  2.46955366  3.50412942 32.32801202
  2.09909847  5.11545639 11.59641641  0.08857442]
test_accu: 0.75  test loss: -7.399089217185974
test_accu: 0.6166666666666667  test loss: -6.441010028123856
test_accu: 0.7833333333333333  test loss: -7.427319794893265
test_accu: 0.65  test loss: -6.410510405898094
test_accu: 0.65  test loss: -6.267199218273163
test_accu: 0.7833333333333333  test loss: -7.5494765639305115
test_accu: 0.6333333333333333  test loss: -6.240287721157074
test_accu: 0.8166666666666667  test loss: -7.999426782131195
test_accu: 0.6666666666666666  test loss: -6.468034595251083
test_accu: 0.8166666666666667  test loss: -8.063801229000092


100%|███████████████████████████████████████████| 10/10 [03:02<00:00, 18.23s/it]

 
Avg Training Stats after 10 global rounds:
Training Loss : -0.6451344507477867
Train Accuracy: 81.67% 



In [8]:
# Test inference after completion of training
test_acc, test_loss = test_inference(args, global_model, test_dataset)

print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

# Saving the objects train_loss and train_accuracy:
file_name = '../save/objects/{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}].pkl'.\
    format(args.dataset, args.model, args.epochs, args.frac, args.iid,
           args.local_ep, args.local_bs)

with open(file_name, 'wb') as f:
    pickle.dump([train_loss, train_accuracy], f)

print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))

print(np.mean(average_time_epochs))
print(np.mean(max_time_epochs))

 
 Results after 10 global rounds of training:
|---- Avg Train Accuracy: 81.67%
|---- Test Accuracy: 73.20%

 Total Run Time: 185.2656
10.437060335765855
34.76089163164503


In [9]:
print(average_time_epochs)
print(max_time_epochs)

[8.407564811354465, 9.334892118546424, 13.482894581917837, 14.831306653150545, 9.117553306114354, 6.297798137677307, 9.216626866356295, 11.06767136851954, 12.501305752386099, 10.112989761635689]
[20.53173557889863, 40.694375192113306, 37.5604733492251, 74.5018921070958, 18.870181518176427, 14.297487385948397, 32.41453638841938, 44.441746476542725, 39.37000624735798, 24.92648207267258]


In [10]:
# PLOTTING (optional)
# import matplotlib
# import matplotlib.pyplot as plt
# matplotlib.use('Agg')

# Plot Loss curve
# plt.figure()
# plt.title('Training Loss vs Communication rounds')
# plt.plot(range(len(train_loss)), train_loss, color='r')
# plt.ylabel('Training loss')
# plt.xlabel('Communication Rounds')
# plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_loss.png'.
#             format(args.dataset, args.model, args.epochs, args.frac,
#                    args.iid, args.local_ep, args.local_bs))
#
# # Plot Average Accuracy vs Communication rounds
# plt.figure()
# plt.title('Average Accuracy vs Communication rounds')
# plt.plot(range(len(train_accuracy)), train_accuracy, color='k')
# plt.ylabel('Average Accuracy')
# plt.xlabel('Communication Rounds')
# plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_acc.png'.
#             format(args.dataset, args.model, args.epochs, args.frac,
#                    args.iid, args.local_ep, args.local_bs))


In [11]:
(79.39+73.32+70.74+76.65+77.48)/5

75.516

In [12]:
(100/30+15*15/60+800/60)/5*72

294.00000000000006

In [13]:
214.0625/15/(2*(1-13.75/15))

85.62499999999997

In [14]:
0.5 * 95.625 + 0.25 * 100.625 + 0.25 *105.625

99.375

In [15]:
0.5  *  25.3125 + 0.25  *51.75 + 0.25  *314

104.09375

In [ ]:
(100/30+15*15/60)/5*9

In [1]:
13.75*13.75/15/(2*(1-13.75/15))

75.62499999999996

In [18]:
0.5 * 15.3125 + 0.25 *27.75 + 0.25 *342

100.09375

In [2]:
0.5*10+0.25*15+ 0.25*20

13.75